In [3]:
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
path = "G:/Shared drives/TriplegangersGroom_ext/Groom_INTERNAL/*/maya/base_delta/images/*/contactSheet.png"
print(path)

G:/Shared drives/TriplegangersGroom_ext/Groom_INTERNAL/*/maya/base_delta/images/*/contactSheet.png


In [23]:
contactSheets = [x.replace(os.sep,'/') for x in glob(path)]
cs_splits = [x.split('/') for x in contactSheets]
cs_splitspds = pd.Series([x.split('/') for x in contactSheets])

df = pd.DataFrame(cs_splits)
df['name'] = df[4]

# unique in column
# print(df['name'].unique())

# unique+count in column
# print(df[4].value_counts())


contactSheetspds = pd.Series([x.replace(os.sep,'/').split('/') for x in glob(path)], name="path")
# contactSheetspds
df = pd.DataFrame(contactSheetspds.tolist())
df
# x = pd.Series([x.replace(os.sep,'/').split('/') for x in glob(path)])
# for y in x:
#     print(y.split('/'))

,0,1,2,3,4,5,6,7,8,9
0,G:,Shared drives,TriplegangersGroom_ext,Groom_INTERNAL,CamilaMazurek,maya,base_delta,images,dh_noise_0.1freq_1mag_0.5corr_dh_noise_13freq_...,contactSheet.png
1,G:,Shared drives,TriplegangersGroom_ext,Groom_INTERNAL,CamilaMazurek,maya,base_delta,images,dh_noise_0.1freq_2mag_0.5corr_dh_noise_13freq_...,contactSheet.png
2,G:,Shared drives,TriplegangersGroom_ext,Groom_INTERNAL,CamilaMazurek,maya,base_delta,images,dh_noise_0.1freq_3mag_0.5corr_dh_noise_13freq_...,contactSheet.png
3,G:,Shared drives,TriplegangersGroom_ext,Groom_INTERNAL,CamilaMazurek,maya,base_delta,images,dh_noise_0.1freq_5mag_0.5corr_dh_noise_13freq_...,contactSheet.png
4,G:,Shared drives,TriplegangersGroom_ext,Groom_INTERNAL,CamilaMazurek,maya,base_delta,images,dh_noise_0.1freq_8mag_0.5corr_dh_noise_13freq_...,contactSheet.png
...,...,...,...,...,...,...,...,...,...,...
216,G:,Shared drives,TriplegangersGroom_ext,Groom_INTERNAL,SimonYuen,maya,base_delta,images,dh_exp_gScale_1.1_dh_exp_gScale_4.1_dh_coil_3....,contactSheet.png
217,G:,Shared drives,TriplegangersGroom_ext,Groom_INTERNAL,SimonYuen,maya,base_delta,images,dh_exp_gScale_1.1_dh_exp_gScale_4.1_dh_coil_4....,contactSheet.png
218,G:,Shared drives,TriplegangersGroom_ext,Groom_INTERNAL,SimonYuen,maya,base_delta,images,dh_exp_gScale_1.1_dh_exp_gScale_4.1_dh_coil_5....,contactSheet.png
219,G:,Shared drives,TriplegangersGroom_ext,Groom_INTERNAL,SimonYuen,maya,base_delta,images,dh_exp_gScale_1.1_dh_exp_gScale_4.1_dh_coil_13...,contactSheet.png
